# Imports, Configurables and Functions

In [ ]:
# Install kagglehub
!pip install kagglehub

# Import necessary libraries
import os
import kagglehub
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torchvision import transforms, models

In [ ]:
# Configurable constants
IMAGE_SIZE = (256, 256)      # Desired image size (e.g., 64x64, 256x256)
REDUCTION_RATIO = 0.99       # Fraction of the dataset to keep
BATCH_SIZE = 64              # Batch size for training and validation
NUM_EPOCHS = 40              # Number of training epochs
LEARNING_RATE = 0.0001       # Learning rate for the optimizer
PATIENCE = 5                 # Patience for early stopping

In [ ]:
# Verify GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Download the UTKFace dataset
path = kagglehub.dataset_download("jangedoo/utkface-new")
data_dir = os.path.join(path, "UTKFace")

In [ ]:
# Preprocess the dataset by extracting image file paths and corresponding age labels.
def preprocess_dataset(data_dir):
    # Get list of image files
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]
    data = []

    # Extract age from filename and create a DataFrame
    for file in image_files:
        try:
            age = int(file.split("_")[0])  # Filename format: age_gender_race_date.jpg
            data.append({"file_path": os.path.join(data_dir, file), "age": age})
        except ValueError:
            print(f"Skipping file: {file} (invalid format)")
    return pd.DataFrame(data)

In [ ]:
# Reduce the dataset size using stratified sampling based on age groups.
def reduce_dataset(df, reduction_ratio, age_bins):
    # Create age groups
    df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=False, right=False)

    # Filter out age groups with fewer than 2 samples
    group_counts = df['age_group'].value_counts()
    valid_groups = group_counts[group_counts >= 2].index
    df = df[df['age_group'].isin(valid_groups)].reset_index(drop=True)

    # Adjust reduction ratio if necessary
    adjusted_ratio = max(reduction_ratio, 2 / len(df))

    # Perform stratified sampling
    sss = StratifiedShuffleSplit(n_splits=1, test_size=1 - adjusted_ratio, random_state=42)
    for train_index, _ in sss.split(df, df['age_group']):
        df_reduced = df.iloc[train_index].reset_index(drop=True)
    return df_reduced

In [ ]:
# Split the reduced dataset into training, validation, and test sets with stratification.
def split_dataset(df_reduced):
    # Ensure each age group has at least two members
    group_counts = df_reduced['age_group'].value_counts()
    valid_groups = group_counts[group_counts >= 2].index
    df_reduced = df_reduced[df_reduced['age_group'].isin(valid_groups)].reset_index(drop=True)

    # Split into training and test sets
    train_df, test_df = train_test_split(
        df_reduced, test_size=0.3, random_state=42, stratify=df_reduced['age_group'])

    # Split the test set into validation and test sets
    val_df, test_df = train_test_split(
        test_df, test_size=0.5, random_state=42, stratify=test_df['age_group'])

    # Drop the temporary 'age_group' column
    train_df = train_df.drop(columns=['age_group'])
    val_df = val_df.drop(columns=['age_group'])
    test_df = test_df.drop(columns=['age_group'])

    return train_df, val_df, test_df

In [ ]:
# Load and preprocess the dataset
df = preprocess_dataset(data_dir)
print(f"Total images: {len(df)}")

# Define age bins for stratification
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 117]

# Reduce the dataset size
df_reduced = reduce_dataset(df, REDUCTION_RATIO, age_bins)
print(f"Reduced dataset size: {len(df_reduced)}")

# Split the dataset into training, validation, and test sets
train_df, val_df, test_df = split_dataset(df_reduced)

# Print dataset sizes
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

Total images: 23708
Reduced dataset size: 23470
Training set size: 16429
Validation set size: 3520
Test set size: 3521


In [ ]:
# Transformations for training data with data augmentation
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],   # Normalization parameters
                         std=[0.229, 0.224, 0.225]),
])

# Transformations for validation and test data without augmentation
val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],    # Same normalization parameters
                         std=[0.229, 0.224, 0.225]),
])

# Age Estimation CNN

In [ ]:
# Custom Dataset for loading UTKFace images and their corresponding age labels.
class UTKFaceDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        # Return the total number of samples
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get the image file path and age label
        row = self.dataframe.iloc[idx]
        image = Image.open(row["file_path"]).convert("RGB")
        label = row["age"]

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        # Return image and label as a tensor
        return image, torch.tensor(label, dtype=torch.float32)

In [ ]:
# Create datasets
train_dataset = UTKFaceDataset(train_df, transform=train_transform)
val_dataset = UTKFaceDataset(val_df, transform=val_transform)
test_dataset = UTKFaceDataset(test_df, transform=val_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=4, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# A CNN model for age estimation from images.
class AgeEstimationCNN(nn.Module):
    def __init__(self):
        super(AgeEstimationCNN, self).__init__()
        # Convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the size of the input for the first fully connected layer
        fc_input_size = 512 * (IMAGE_SIZE[0] // (2**5)) * (IMAGE_SIZE[1] // (2**5))

        # Fully connected layers with dropout
        self.fc1 = nn.Linear(fc_input_size, 1024)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(1024, 1)  # Output layer for age regression

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and pooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # Output size: H/2, W/2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # Output size: H/4, W/4
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Output size: H/8, W/8
        x = self.pool(F.relu(self.bn4(self.conv4(x))))  # Output size: H/16, W/16
        x = self.pool(F.relu(self.bn5(self.conv5(x))))  # Output size: H/32, W/32

        # Flatten the tensor for fully connected layers
        x = x.view(x.size(0), -1)

        # Apply fully connected layers with ReLU and dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)  # Output the estimated age

        return x

In [ ]:
# Initialize the model
model = AgeEstimationCNN().to(device)

# Initialize weights of the model using appropriate initialization methods.
def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)

# Apply weight initialization
model.apply(initialize_weights)

# Define loss function (Mean Squared Error for regression)
criterion = nn.MSELoss()

# Define optimizer (Adam optimizer)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler to adjust learning rate during training
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Early stopping parameters
best_mae = float('inf')  # Initialize best Mean Absolute Error
patience = PATIENCE      # Number of epochs to wait before early stopping
trigger_times = 0        # Counter for early stopping

In [ ]:
# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()  # Set model to training mode
    total_loss = 0.0

    # Iterate over training data
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)  # Reshape labels

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Step the learning rate scheduler
    scheduler.step()

    # Calculate average training loss
    avg_train_loss = total_loss / len(train_loader)

    # Validation loop
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0.0
    total_mae = 0.0  # Mean Absolute Error

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).float().unsqueeze(1)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

            # Calculate Mean Absolute Error
            mae = torch.abs(outputs - labels).mean().item()
            total_mae += mae

    # Calculate average validation loss and MAE
    avg_val_loss = total_val_loss / len(val_loader)
    avg_mae = total_mae / len(val_loader)

    # Print training and validation statistics
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, "
          f"Val MAE: {avg_mae:.4f}")

    # Early stopping check
    if avg_mae < best_mae:
        best_mae = avg_mae
        trigger_times = 0
        # Save the best model weights
        torch.save(model.state_dict(), 'age_estimation_model.pth')
        print("Validation MAE decreased, saving model...")
    else:
        trigger_times += 1
        print(f"Validation MAE did not improve. Trigger times: {trigger_times}")
        if trigger_times >= patience:
            print("Early stopping!")
            break

Epoch 1/40, Train Loss: 263.5961, Val Loss: 166.1904, Val MAE: 10.0642
Validation MAE decreased, saving model...
Epoch 2/40, Train Loss: 173.4490, Val Loss: 133.7179, Val MAE: 8.6674
Validation MAE decreased, saving model...
Epoch 3/40, Train Loss: 144.9623, Val Loss: 125.6387, Val MAE: 8.4578
Validation MAE decreased, saving model...
Epoch 4/40, Train Loss: 132.0370, Val Loss: 106.2599, Val MAE: 7.6829
Validation MAE decreased, saving model...
Epoch 5/40, Train Loss: 121.5501, Val Loss: 97.4598, Val MAE: 7.2920
Validation MAE decreased, saving model...
Epoch 6/40, Train Loss: 114.4948, Val Loss: 111.1744, Val MAE: 7.7012
Validation MAE did not improve. Trigger times: 1
Epoch 7/40, Train Loss: 106.5586, Val Loss: 95.5599, Val MAE: 7.1966
Validation MAE decreased, saving model...
Epoch 8/40, Train Loss: 93.7656, Val Loss: 85.0309, Val MAE: 6.8025
Validation MAE decreased, saving model...
Epoch 9/40, Train Loss: 88.9150, Val Loss: 84.5422, Val MAE: 6.7085
Validation MAE decreased, saving

In [ ]:
# Load the best model weights
model.load_state_dict(torch.load('age_estimation_model.pth'))

# Set model to evaluation mode
model.eval()
total_mae_test = 0.0

# Disable gradient calculation
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        # Forward pass
        outputs = model(images)

        # Calculate Mean Absolute Error
        mae = torch.abs(outputs - labels).mean().item()
        total_mae_test += mae

# Print the final test MAE
print(f"Test Mean Absolute Error (MAE): {total_mae_test / len(test_loader):.4f}")

<ipython-input-25-c4859a848353>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('age_estimation_model.pth'))


Test Mean Absolute Error (MAE): 6.5153


## ResNet Transfer Learning

In [ ]:
!pip install kagglehub

# ================================================
# Age Estimation using ResNet18 with Transfer Learning
# ================================================

# ----------------------------
# Step 1: Import Libraries
# ----------------------------

import os
import kagglehub
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torchvision import transforms, models
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.optim as optim

# ----------------------------
# Step 2: Define Configurable Constants
# ----------------------------

# Image size expected by ResNet18
IMAGE_SIZE = (224, 224)

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Hyperparameters
REDUCTION_RATIO = 1.0          # Use full dataset
BATCH_SIZE = 64                # Batch size for training and validation
NUM_EPOCHS = 10                # Increased number of training epochs
LEARNING_RATE = 1e-4           # Learning rate for the optimizer
PATIENCE = 10                  # Increased patience for early stopping

# ----------------------------
# Step 3: Download and Preprocess the Dataset
# ----------------------------

# Download the UTKFace dataset using kagglehub
path = kagglehub.dataset_download("jangedoo/utkface-new")
data_dir = os.path.join(path, "UTKFace")

# Function to preprocess dataset: extract image paths and age labels
def preprocess_dataset(data_dir):
    # List all image files in the directory
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]
    data = []

    # Extract age from filename and create a DataFrame
    for file in image_files:
        try:
            age = int(file.split("_")[0])  # Filename format: age_gender_race_date.jpg
            data.append({"file_path": os.path.join(data_dir, file), "age": age})
        except ValueError:
            print(f"Skipping file: {file} (invalid format)")
    return pd.DataFrame(data)

# Load and preprocess the dataset
df = preprocess_dataset(data_dir)
print(f"Total images: {len(df)}")

# Function to reduce dataset size using stratified sampling based on age groups
def reduce_dataset(df, reduction_ratio, age_bins):
    if reduction_ratio < 1.0:
        # Create age groups
        df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=False, right=False)

        # Filter out age groups with fewer than 2 samples
        group_counts = df['age_group'].value_counts()
        valid_groups = group_counts[group_counts >= 2].index
        df = df[df['age_group'].isin(valid_groups)].reset_index(drop=True)

        # Perform stratified sampling
        sss = StratifiedShuffleSplit(n_splits=1, test_size=1 - reduction_ratio, random_state=42)
        for train_index, _ in sss.split(df, df['age_group']):
            df_reduced = df.iloc[train_index].reset_index(drop=True)
        return df_reduced
    else:
        return df

# Define age bins for stratification
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 117]

# Reduce the dataset size
df_reduced = reduce_dataset(df, REDUCTION_RATIO, age_bins)
print(f"Reduced dataset size: {len(df_reduced)}")

# Function to split the dataset into training, validation, and test sets
def split_dataset(df_reduced, age_bins):
    # Create age groups
    df_reduced['age_group'] = pd.cut(df_reduced['age'], bins=age_bins, labels=False, right=False)

    # Split into training and temp (validation + test)
    train_df, temp_df = train_test_split(
        df_reduced, test_size=0.3, random_state=42, stratify=df_reduced['age_group'])

    # Split temp into validation and test
    val_df, test_df = train_test_split(
        temp_df, test_size=0.5, random_state=42, stratify=temp_df['age_group'])

    # Drop the temporary 'age_group' column
    train_df = train_df.drop(columns=['age_group'])
    val_df = val_df.drop(columns=['age_group'])
    test_df = test_df.drop(columns=['age_group'])

    return train_df, val_df, test_df

# Split the dataset
train_df, val_df, test_df = split_dataset(df_reduced, age_bins)

# Print dataset sizes
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

# ----------------------------
# Step 4: Define Data Augmentation and Transformation
# ----------------------------

# Define data augmentation and normalization for training
# Just normalization for validation and testing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(IMAGE_SIZE[0]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],   # ImageNet mean
                             [0.229, 0.224, 0.225])   # ImageNet std
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_SIZE[0]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_SIZE[0]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

# ----------------------------
# Step 5: Create Custom Dataset Class
# ----------------------------

class UTKFaceDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row["file_path"]).convert("RGB")
        age = row["age"]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(age, dtype=torch.float32)

# ----------------------------
# Step 6: Create DataLoaders
# ----------------------------

# Create datasets
image_datasets = {
    'train': UTKFaceDataset(train_df, transform=data_transforms['train']),
    'val': UTKFaceDataset(val_df, transform=data_transforms['val']),
    'test': UTKFaceDataset(test_df, transform=data_transforms['test']),
}

# Create DataLoaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
    'val': DataLoader(image_datasets['val'], batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True),
    'test': DataLoader(image_datasets['test'], batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True),
}

# ----------------------------
# Step 7: Define the Model
# ----------------------------

# Load the pretrained ResNet18 model
model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layer4 and fc for fine-tuning
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Modify the final fully connected layer for regression
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1)
)

# Move the model to the appropriate device
model = model.to(DEVICE)

# ----------------------------
# Step 8: Define Loss Function, Optimizer, and Scheduler
# ----------------------------

# Use Mean Squared Error Loss for regression
criterion = nn.MSELoss()

# Update the optimizer to include parameters that are being fine-tuned
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE, weight_decay=1e-5)

# Define learning rate scheduler to reduce LR on plateau
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# ----------------------------
# Step 9: Training and Validation Loop
# ----------------------------

def train_model(model, criterion, optimizer, scheduler, num_epochs=30, patience=10):
    best_mae = float('inf')
    best_model_wts = None
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_mae = 0.0
            running_samples = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE).unsqueeze(1)  # Shape: [batch_size, 1]

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    mae = torch.abs(outputs - labels).sum().item()

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_mae += mae
                running_samples += inputs.size(0)

            epoch_loss = running_loss / running_samples
            epoch_mae = running_mae / running_samples

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} MAE: {epoch_mae:.4f}")

            # Step the scheduler based on validation loss
            if phase == 'val':
                scheduler.step(epoch_loss)

                # Early stopping
                if epoch_mae < best_mae:
                    best_mae = epoch_mae
                    best_model_wts = model.state_dict()
                    patience_counter = 0
                    torch.save(best_model_wts, 'best_resnet18_age_estimation.pth')
                    print("Validation MAE improved. Model saved.")
                else:
                    patience_counter += 1
                    print(f"No improvement in Validation MAE. Patience counter: {patience_counter}/{patience}")

        print()

        # Check early stopping condition
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    print(f"Best Validation MAE: {best_mae:.4f}")

    # Load best model weights
    if best_model_wts is not None:
        model.load_state_dict(best_model_wts)

    return model

# ----------------------------
# Step 10: Train the Model
# ----------------------------

# Train the model
model = train_model(model, criterion, optimizer, scheduler, num_epochs=NUM_EPOCHS, patience=PATIENCE)

# ----------------------------
# Step 11: Evaluate the Model on Test Set
# ----------------------------

def evaluate_model(model, dataloader):
    model.eval()
    total_mae = 0.0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE).unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            mae = torch.abs(outputs - labels).sum().item()

            total_mae += mae
            total_samples += inputs.size(0)

    final_mae = total_mae / total_samples
    return final_mae

# Evaluate on test set
test_mae = evaluate_model(model, dataloaders['test'])
print(f"Final Test MAE: {test_mae:.4f}")

# ----------------------------
# Step 12: Visualize Predictions
# ----------------------------

def visualize_predictions(model, dataloader, num_samples=100):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE).unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

            if len(predictions) >= num_samples:
                break

    predictions = np.array(predictions[:num_samples]).flatten()
    actuals = np.array(actuals[:num_samples]).flatten()

    plt.figure(figsize=(8, 8))
    plt.scatter(actuals, predictions, alpha=0.6)
    plt.plot([0, 120], [0, 120], 'r--')  # Diagonal line
    plt.xlabel('Actual Age')
    plt.ylabel('Predicted Age')
    plt.title('Actual vs. Predicted Age')
    plt.xlim(0, 120)
    plt.ylim(0, 120)
    plt.grid(True)
    plt.show()

# Visualize predictions
visualize_predictions(model, dataloaders['test'])

# ----------------------------
# Step 13: Save the Final Model
# ----------------------------

# Save the model architecture and weights
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'resnet18_age_estimation_final.pth')

print("Model saved successfully.")

Using device: cuda
Total images: 23708
Reduced dataset size: 23708
Training set size: 16595
Validation set size: 3556
Test set size: 3557


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNe

Epoch 1/10
----------


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


KeyboardInterrupt: 

# Gender Classificiation

In [7]:
# Install kagglehub
!pip install kagglehub

# Import necessary libraries
import os
import kagglehub
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler

SystemError: <built-in function isinstance> returned a result with an exception set

In [ ]:
# Configurable constants
IMAGE_SIZE = (256, 256)      # Desired image size (e.g., 64x64, 256x256)
REDUCTION_RATIO = 0.41       # Fraction of the dataset to keep
BATCH_SIZE = 64              # Batch size for training and validation
NUM_EPOCHS = 5              # Number of training epochs
LEARNING_RATE = 0.0001       # Learning rate for the optimizer
PATIENCE = 5                 # Patience for early stopping

In [ ]:
# Verify GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Download the UTKFace dataset
path = kagglehub.dataset_download("jangedoo/utkface-new")
data_dir = os.path.join(path, "UTKFace")

In [ ]:
# Preprocess the dataset by extracting image file paths and corresponding gender labels.
def preprocess_dataset(data_dir):
    # Get list of image files
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]
    data = []

    # Extract gender from filename and create a DataFrame
    for file in image_files:
        try:
            gender = int(file.split("_")[1])  # Filename format: age_gender_race_date.jpg
            data.append({"file_path": os.path.join(data_dir, file), "gender": gender})
        except ValueError:
            print(f"Skipping file: {file} (invalid format)")
    return pd.DataFrame(data)

In [ ]:
# Reduce the dataset size using stratified sampling based on gender groups.
def reduce_dataset(df, reduction_ratio, gender_bins):
    # Create gender groups
    df['gender_group'] = pd.cut(df['gender'], bins=gender_bins, labels=False, right=False)

    # Filter out gender groups with fewer than 2 samples
    group_counts = df['gender_group'].value_counts()
    valid_groups = group_counts[group_counts >= 2].index
    df = df[df['gender_group'].isin(valid_groups)].reset_index(drop=True)

    # Adjust reduction ratio if necessary
    adjusted_ratio = max(reduction_ratio, 2 / len(df))

    # Perform stratified sampling
    sss = StratifiedShuffleSplit(n_splits=1, test_size=1 - adjusted_ratio, random_state=42)
    for train_index, _ in sss.split(df, df['gender_group']):
        df_reduced = df.iloc[train_index].reset_index(drop=True)
    return df_reduced

In [ ]:
# Split the reduced dataset into training, validation, and test sets with stratification.
def split_dataset(df_reduced):
    # Ensure each gender group has at least two members
    group_counts = df_reduced['gender_group'].value_counts()
    valid_groups = group_counts[group_counts >= 2].index
    df_reduced = df_reduced[df_reduced['gender_group'].isin(valid_groups)].reset_index(drop=True)

    # Split into training and test sets
    train_df, test_df = train_test_split(
        df_reduced, test_size=0.3, random_state=42, stratify=df_reduced['gender_group'])

    # Split the test set into validation and test sets
    val_df, test_df = train_test_split(
        test_df, test_size=0.5, random_state=42, stratify=test_df['gender_group'])

    # Drop the temporary 'gender_group' column
    train_df = train_df.drop(columns=['gender_group'])
    val_df = val_df.drop(columns=['gender_group'])
    test_df = test_df.drop(columns=['gender_group'])

    return train_df, val_df, test_df

In [ ]:
# Load and preprocess the dataset
df = preprocess_dataset(data_dir)
print(f"Total images: {len(df)}")

# Define gender bins for stratification
gender_bins = [0, 1]

# Reduce the dataset size
df_reduced = reduce_dataset(df, REDUCTION_RATIO, gender_bins)
print(f"Reduced dataset size: {len(df_reduced)}")

# Split the dataset into training, validation, and test sets
train_df, val_df, test_df = split_dataset(df_reduced)

# Print dataset sizes
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

In [ ]:
# Transformations for training data with data augmentation
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],   # Normalization parameters
                         std=[0.229, 0.224, 0.225]),
])

# Transformations for validation and test data without augmentation
val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],    # Same normalization parameters
                         std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create datasets
train_dataset = UTKFaceDataset(train_df, transform=train_transform)
val_dataset = UTKFaceDataset(val_df, transform=val_transform)
test_dataset = UTKFaceDataset(test_df, transform=val_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=4, pin_memory=True)

In [ ]:
# A CNN model for gender estimation from images.
class ImprovedGenderEstimationCNN(nn.Module):
    def __init__(self):
        super(ImprovedGenderEstimationCNN, self).__init__()
        # Convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the size of the input for the first fully connected layer
        fc_input_size = 512 * (IMAGE_SIZE[0] // (2**5)) * (IMAGE_SIZE[1] // (2**5))

        # Fully connected layers with dropout
        self.fc1 = nn.Linear(fc_input_size, 1024)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(1024, 1)  # Output layer for gender regression

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and pooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # Output size: H/2, W/2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # Output size: H/4, W/4
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Output size: H/8, W/8
        x = self.pool(F.relu(self.bn4(self.conv4(x))))  # Output size: H/16, W/16
        x = self.pool(F.relu(self.bn5(self.conv5(x))))  # Output size: H/32, W/32

        # Flatten the tensor for fully connected layers
        x = x.view(x.size(0), -1)

        # Apply fully connected layers with ReLU and dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)  # Output the estimated gender

        return x

In [ ]:
# Initialize the model
model = ImprovedGenderEstimationCNN().to(device)

# Initialize weights of the model using appropriate initialization methods.
def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)

# Apply weight initialization
model.apply(initialize_weights)

# Define loss function (Mean Squared Error for regression)
criterion = nn.MSELoss()

# Define optimizer (Adam optimizer)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler to adjust learning rate during training
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()  # Set model to training mode
    total_loss = 0.0

    # Iterate over training data
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)  # Reshape labels

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Step the learning rate scheduler
    scheduler.step()

    # Calculate average training loss
    avg_train_loss = total_loss / len(train_loader)

    # Validation loop
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0.0
    total_mae = 0.0  # Mean Absolute Error

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).float().unsqueeze(1)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

            # Calculate Mean Absolute Error
            mae = torch.abs(outputs - labels).mean().item()
            total_mae += mae

    # Calculate average validation loss and MAE
    avg_val_loss = total_val_loss / len(val_loader)
    avg_mae = total_mae / len(val_loader)

    # Print training and validation statistics
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, "
          f"Val MAE: {avg_mae:.4f}")

    # Early stopping check
    if avg_mae < best_mae:
        best_mae = avg_mae
        trigger_times = 0
        # Save the best model weights
        torch.save(model.state_dict(), 'best_model.pth')
        print("Validation MAE decreased, saving model...")
    else:
        trigger_times += 1
        print(f"Validation MAE did not improve. Trigger times: {trigger_times}")
        if trigger_times >= patience:
            print("Early stopping!")
            break

In [ ]:
# Load the best model weights
model.load_state_dict(torch.load('best_model.pth'))

# Set model to evaluation mode
model.eval()
total_mae_test = 0.0

# Disable gradient calculation
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device).float().unsqueeze(1)

        # Forward pass
        outputs = model(images)

        # Calculate Mean Absolute Error
        mae = torch.abs(outputs - labels).mean().item()
        total_mae_test += mae

# Print the final test MAE
print(f"Test Mean Absolute Error (MAE): {total_mae_test / len(test_loader):.4f}")

# Fixed Gender Classification

In [4]:
# Install kagglehub if not already installed
!pip install kagglehub

# Import necessary libraries
import os
import kagglehub
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler

# Configurable constants
IMAGE_SIZE = (256, 256)      # Desired image size (e.g., 64x64, 256x256)
REDUCTION_RATIO = 0.99       # Fraction of the dataset to keep
BATCH_SIZE = 64              # Batch size for training and validation
NUM_EPOCHS = 20              # Number of training epochs
LEARNING_RATE = 0.0001       # Learning rate for the optimizer
PATIENCE = 5                 # Patience for early stopping

# Verify GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Download the UTKFace dataset
path = kagglehub.dataset_download("jangedoo/utkface-new")
data_dir = os.path.join(path, "UTKFace")

# Preprocess the dataset by extracting image file paths and corresponding labels.
def preprocess_dataset(data_dir):
    # Get list of image files
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]
    data = []

    # Filename format: age_gender_race_date.jpg
    # According to UTKFace: gender=0 is male, gender=1 is female.
    for file in image_files:
        try:
            parts = file.split("_")
            age = int(parts[0])
            gender = int(parts[1])  # 0=male, 1=female
            data.append({"file_path": os.path.join(data_dir, file), "age": age, "gender": gender})
        except ValueError:
            print(f"Skipping file: {file} (invalid format)")
    return pd.DataFrame(data)

# Reduce the dataset size using stratified sampling based on age groups.
def reduce_dataset(df, reduction_ratio, age_bins):
    # Create age groups
    df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=False, right=False)

    # Filter out age groups with fewer than 2 samples
    group_counts = df['age_group'].value_counts()
    valid_groups = group_counts[group_counts >= 2].index
    df = df[df['age_group'].isin(valid_groups)].reset_index(drop=True)

    # Adjust reduction ratio if necessary
    adjusted_ratio = max(reduction_ratio, 2 / len(df))

    # Perform stratified sampling on age_group
    sss = StratifiedShuffleSplit(n_splits=1, test_size=1 - adjusted_ratio, random_state=42)
    for train_index, _ in sss.split(df, df['age_group']):
        df_reduced = df.iloc[train_index].reset_index(drop=True)
    return df_reduced

# Split the reduced dataset into training, validation, and test sets with stratification.
def split_dataset(df_reduced):
    # Ensure each age group has at least two members
    group_counts = df_reduced['age_group'].value_counts()
    valid_groups = group_counts[group_counts >= 2].index
    df_reduced = df_reduced[df_reduced['age_group'].isin(valid_groups)].reset_index(drop=True)

    # Split into training and test sets stratified by age_group
    train_df, test_df = train_test_split(
        df_reduced, test_size=0.3, random_state=42, stratify=df_reduced['age_group'])

    # Split the test set into validation and test sets
    val_df, test_df = train_test_split(
        test_df, test_size=0.5, random_state=42, stratify=test_df['age_group'])

    # Drop the temporary 'age_group' column
    train_df = train_df.drop(columns=['age_group'])
    val_df = val_df.drop(columns=['age_group'])
    test_df = test_df.drop(columns=['age_group'])

    return train_df, val_df, test_df

# Load and preprocess the dataset
df = preprocess_dataset(data_dir)
print(f"Total images: {len(df)}")

# Define age bins for stratification (example: 0-10,10-20,...,90-100)
age_bins = range(0, 101, 10)

# Reduce the dataset size
df_reduced = reduce_dataset(df, REDUCTION_RATIO, age_bins)
print(f"Reduced dataset size: {len(df_reduced)}")

# Split the dataset into training, validation, and test sets
train_df, val_df, test_df = split_dataset(df_reduced)

# Print dataset sizes
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

class UTKFaceDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['file_path']).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Gender: 0=male, 1=female
        label = row['gender']
        return image, label

# Transformations for training data with data augmentation
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Transformations for validation and test data without augmentation
val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = UTKFaceDataset(train_df, transform=train_transform)
val_dataset = UTKFaceDataset(val_df, transform=val_transform)
test_dataset = UTKFaceDataset(test_df, transform=val_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# A CNN model for gender classification
class ImprovedGenderEstimationCNN(nn.Module):
    def __init__(self):
        super(ImprovedGenderEstimationCNN, self).__init__()
        # Convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the size of the input for the first fully connected layer
        fc_input_size = 512 * (IMAGE_SIZE[0] // (2**5)) * (IMAGE_SIZE[1] // (2**5))

        # Fully connected layers with dropout
        self.fc1 = nn.Linear(fc_input_size, 1024)
        self.dropout = nn.Dropout(p=0.5)
        # Output layer for binary classification (0=male, 1=female)
        self.fc2 = nn.Linear(1024, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = x.view(x.size(0), -1)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)  # Logits for binary classification
        return x

# Initialize the model
model = ImprovedGenderEstimationCNN().to(device)

# Initialize weights
def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

model.apply(initialize_weights)

# For binary classification, use BCEWithLogitsLoss
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

best_val_loss = float('inf')
trigger_times = 0
patience = PATIENCE

for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Compute training accuracy
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()
        correct = (preds == labels.float().unsqueeze(1)).sum().item()
        total_correct += correct
        total_samples += labels.size(0)

    scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    train_acc = total_correct / total_samples

    # Validation loop
    model.eval()
    total_val_loss = 0.0
    total_val_correct = 0
    val_samples = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            total_val_loss += loss.item()

            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()
            correct = (preds == labels.float().unsqueeze(1)).sum().item()
            total_val_correct += correct
            val_samples += labels.size(0)

    avg_val_loss = total_val_loss / len(val_loader)
    val_acc = total_val_correct / val_samples

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Early stopping based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print("Validation loss decreased, saving model...")
    else:
        trigger_times += 1
        print(f"Validation loss did not improve. Trigger times: {trigger_times}")
        if trigger_times >= patience:
            print("Early stopping!")
            break

# Load the best model weights
model.load_state_dict(torch.load('best_model.pth'))

# Test evaluation
model.eval()
test_correct = 0
test_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()
        correct = (preds == labels.float().unsqueeze(1)).sum().item()
        test_correct += correct
        test_samples += labels.size(0)

test_acc = test_correct / test_samples
print(f"Test Accuracy: {test_acc:.4f}")

Using device: cuda
Total images: 23708
Reduced dataset size: 23439
Training set size: 16407
Validation set size: 3516
Test set size: 3516
Epoch 1/20, Train Loss: 0.7510, Train Acc: 0.7103, Val Loss: 0.4348, Val Acc: 0.8100
Validation loss decreased, saving model...
Epoch 2/20, Train Loss: 0.4120, Train Acc: 0.8152, Val Loss: 0.3519, Val Acc: 0.8521
Validation loss decreased, saving model...
Epoch 3/20, Train Loss: 0.3543, Train Acc: 0.8403, Val Loss: 0.3131, Val Acc: 0.8555
Validation loss decreased, saving model...
Epoch 4/20, Train Loss: 0.3201, Train Acc: 0.8595, Val Loss: 0.2874, Val Acc: 0.8706
Validation loss decreased, saving model...
Epoch 5/20, Train Loss: 0.2972, Train Acc: 0.8704, Val Loss: 0.2701, Val Acc: 0.8786
Validation loss decreased, saving model...
Epoch 6/20, Train Loss: 0.2817, Train Acc: 0.8782, Val Loss: 0.2696, Val Acc: 0.8817
Validation loss decreased, saving model...
Epoch 7/20, Train Loss: 0.2718, Train Acc: 0.8835, Val Loss: 0.2774, Val Acc: 0.8788
Validatio

<ipython-input-4-cbc5117a0611>:301: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Accuracy: 0.9110


# Live Example

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

# Copy the class definition here:
class AgeEstimationCNN(nn.Module):
    def __init__(self):
        super(AgeEstimationCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Adjust this if your IMAGE_SIZE changed, but from the code you provided it was (256,256)
        fc_input_size = 512 * (256 // (2**5)) * (256 // (2**5))

        self.fc1 = nn.Linear(fc_input_size, 1024)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(1024, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Load model
model = AgeEstimationCNN()
model.load_state_dict(torch.load('age_estimation_model.pth', map_location='cpu'))
model.eval()

# Define transforms (must match training)
input_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load and preprocess the input image
image = Image.open('picture.jpg').convert('RGB')
input_tensor = input_transforms(image).unsqueeze(0)

# Run inference
with torch.no_grad():
    output = model(input_tensor)

predicted_age = output.item()
print("Predicted Age:", predicted_age)

<ipython-input-32-a677927958bd>:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('age_estimation_model.pth', map_location='cpu'))


Predicted Age: 25.045183181762695
